# Practical Seismology Applications

This notebook demonstrates practical applications of the `seisray` package for common seismological problems and research scenarios.

## Learning Objectives
- Apply seisray to realistic seismological problems
- Earthquake location and magnitude estimation
- Regional velocity structure studies
- Station array design and optimization
- Quality control for seismic data

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import sys
import os
from datetime import datetime, timedelta

# Add the parent directory to the path to import seisray
sys.path.append(os.path.dirname(os.getcwd()))

from seisray import (TravelTimeCalculator, RayPathTracer, EarthPlotter,
                     EarthModelManager, CoordinateConverter)

print("Successfully imported seisray package!")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## 1. Earthquake Location Problem

Let's simulate an earthquake location scenario using travel time differences.

In [ ]:
# Define a synthetic earthquake and station network
# Earthquake location (unknown - to be determined)
true_eq_lat = 37.5  # degrees N
true_eq_lon = -122.0  # degrees W (San Francisco Bay Area)
true_eq_depth = 8.0  # km
true_origin_time = datetime(2024, 9, 5, 14, 30, 15)  # Origin time

# Station network (known locations)
stations = {
    'STA1': {'lat': 37.8, 'lon': -122.4, 'name': 'Station 1'},
    'STA2': {'lat': 37.2, 'lon': -121.8, 'name': 'Station 2'},
    'STA3': {'lat': 37.6, 'lon': -121.5, 'name': 'Station 3'},
    'STA4': {'lat': 37.9, 'lon': -121.9, 'name': 'Station 4'},
    'STA5': {'lat': 37.1, 'lon': -122.2, 'name': 'Station 5'},
}

print(f"Synthetic Earthquake Scenario:")
print(f"  True location: {true_eq_lat:.2f}°N, {abs(true_eq_lon):.2f}°W")
print(f"  True depth: {true_eq_depth:.1f} km")
print(f"  Origin time: {true_origin_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\nStation Network:")
for sta_code, sta_info in stations.items():
    print(f"  {sta_code}: {sta_info['lat']:.1f}°N, {abs(sta_info['lon']):.1f}°W")

In [ ]:
# Calculate true distances and generate synthetic arrival times
calc = TravelTimeCalculator('iasp91')
converter = CoordinateConverter()

synthetic_arrivals = {}

print(f"\nSynthetic Arrival Times:")
print(f"{'Station':<8} {'Distance (km)':<12} {'Distance (°)':<12} {'P-time (s)':<12} {'S-time (s)':<12} {'S-P (s)':<10}")
print("-" * 80)

for sta_code, sta_info in stations.items():
    # Calculate distance
    dist_km, dist_deg, azimuth = converter.geographic_to_distance(
        true_eq_lat, true_eq_lon, sta_info['lat'], sta_info['lon']
    )

    # Calculate travel times
    arrivals = calc.calculate_travel_times(true_eq_depth, dist_deg)

    # Find P and S arrivals
    p_time = None
    s_time = None

    for arrival in arrivals:
        if arrival.name == 'P' and p_time is None:
            p_time = arrival.time
        elif arrival.name == 'S' and s_time is None:
            s_time = arrival.time

    # Add some realistic noise (±0.1 seconds)
    noise_p = np.random.normal(0, 0.1)
    noise_s = np.random.normal(0, 0.15)

    p_time_noisy = p_time + noise_p
    s_time_noisy = s_time + noise_s
    sp_time = s_time_noisy - p_time_noisy

    # Calculate absolute arrival times
    p_arrival_time = true_origin_time + timedelta(seconds=p_time_noisy)
    s_arrival_time = true_origin_time + timedelta(seconds=s_time_noisy)

    synthetic_arrivals[sta_code] = {
        'distance_km': dist_km,
        'distance_deg': dist_deg,
        'azimuth': azimuth,
        'p_time': p_time_noisy,
        's_time': s_time_noisy,
        'sp_time': sp_time,
        'p_arrival': p_arrival_time,
        's_arrival': s_arrival_time
    }

    print(f"{sta_code:<8} {dist_km:<12.1f} {dist_deg:<12.2f} {p_time_noisy:<12.2f} {s_time_noisy:<12.2f} {sp_time:<10.2f}")

In [ ]:
# Visualize the earthquake-station geometry
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Plot stations
for sta_code, sta_info in stations.items():
    ax.plot(sta_info['lon'], sta_info['lat'], 'g^', markersize=12, label='Stations' if sta_code == 'STA1' else '')
    ax.annotate(sta_code, (sta_info['lon'], sta_info['lat']),
               xytext=(5, 5), textcoords='offset points', fontsize=10)

# Plot true earthquake location
ax.plot(true_eq_lon, true_eq_lat, 'r*', markersize=20, label='True Earthquake')

# Plot distance circles
colors = ['blue', 'red', 'green', 'purple', 'orange']
for i, (sta_code, sta_info) in enumerate(stations.items()):
    arrival_data = synthetic_arrivals[sta_code]

    # Convert distance to approximate degrees for plotting
    dist_deg_approx = arrival_data['distance_km'] / 111.0  # Rough conversion

    circle = Circle((sta_info['lon'], sta_info['lat']), dist_deg_approx,
                   fill=False, color=colors[i], linewidth=2, alpha=0.7,
                   label=f'{sta_code} ({arrival_data["distance_km"]:.1f} km)')
    ax.add_patch(circle)

ax.set_xlabel('Longitude (degrees)')
ax.set_ylabel('Latitude (degrees)')
ax.set_title('Earthquake Location Problem\nTrue location and station distances')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

## 2. Regional Velocity Structure Study

Let's compare how different Earth models perform for this regional study.

In [ ]:
# Compare different Earth models for regional distances
models = ['iasp91', 'prem', 'ak135']
regional_distances = np.array([10, 20, 30, 40, 50])  # degrees - regional scale
regional_depth = 15  # km - typical crustal earthquake

model_comparison = {}

print(f"Regional Velocity Structure Comparison:")
print(f"Source depth: {regional_depth} km")
print(f"\nP-wave Travel Times (seconds):")
print(f"{'Distance (°)':<12}", end="")
for model in models:
    print(f"{model.upper():<10}", end="")
print()
print("-" * (12 + 10 * len(models)))

for distance in regional_distances:
    print(f"{distance:<12.0f}", end="")

    distance_data = {}
    for model in models:
        calc = TravelTimeCalculator(model)
        arrivals = calc.calculate_travel_times(regional_depth, distance)

        # Find P arrival
        p_time = None
        for arrival in arrivals:
            if arrival.name == 'P':
                p_time = arrival.time
                break

        distance_data[model] = p_time
        print(f"{p_time:<10.2f}", end="")

    model_comparison[distance] = distance_data
    print()

# Calculate residuals relative to IASP91
print(f"\nResiduals relative to IASP91 (seconds):")
print(f"{'Distance (°)':<12}", end="")
for model in models[1:]:  # Skip IASP91
    print(f"{model.upper()}-IASP91:<15}", end="")
print()
print("-" * (12 + 15 * (len(models) - 1)))

for distance in regional_distances:
    print(f"{distance:<12.0f}", end="")

    iasp91_time = model_comparison[distance]['iasp91']
    for model in models[1:]:
        residual = model_comparison[distance][model] - iasp91_time
        print(f"{residual:<15.3f}", end="")
    print()

In [ ]:
# Plot regional travel time curves
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
colors = ['blue', 'red', 'green']

# Travel time curves
ax = axes[0]
for model, color in zip(models, colors):
    times = [model_comparison[dist][model] for dist in regional_distances]
    ax.plot(regional_distances, times, color=color, linewidth=2,
           marker='o', markersize=6, label=model.upper())

ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('P-wave Travel Time (s)')
ax.set_title(f'Regional P-wave Travel Times\n(Depth = {regional_depth} km)')
ax.legend()
ax.grid(True, alpha=0.3)

# Residual plot
ax = axes[1]
for model, color in zip(models[1:], colors[1:]):
    residuals = [model_comparison[dist][model] - model_comparison[dist]['iasp91']
                for dist in regional_distances]
    ax.plot(regional_distances, residuals, color=color, linewidth=2,
           marker='s', markersize=6, label=f'{model.upper()} - IASP91')

ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('Travel Time Residual (s)')
ax.set_title('Model Residuals (relative to IASP91)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary
print(f"\nStatistical Summary of Model Differences:")
for model in models[1:]:
    residuals = [model_comparison[dist][model] - model_comparison[dist]['iasp91']
                for dist in regional_distances]
    print(f"{model.upper()} vs IASP91:")
    print(f"  Mean residual: {np.mean(residuals):.3f} s")
    print(f"  RMS residual: {np.sqrt(np.mean(np.array(residuals)**2)):.3f} s")
    print(f"  Max absolute: {np.max(np.abs(residuals)):.3f} s")

## 3. Station Array Design

Let's design an optimal station array for monitoring local seismicity.

In [ ]:
# Design a circular station array
center_lat = 37.5  # degrees N
center_lon = -122.0  # degrees W
array_radius = 50  # km
n_stations = 8

# Create station array
converter = CoordinateConverter()
array_stations = converter.create_station_array(center_lat, center_lon,
                                               array_radius, n_stations)

print(f"Optimal Station Array Design:")
print(f"  Center: {center_lat:.2f}°N, {abs(center_lon):.2f}°W")
print(f"  Radius: {array_radius} km")
print(f"  Number of stations: {n_stations}")
print(f"\nStation Coordinates:")
print(f"{'Station':<8} {'Latitude':<10} {'Longitude':<11} {'Distance (km)':<13} {'Azimuth (°)':<12}")
print("-" * 65)

array_info = []
for i, (lat, lon) in enumerate(array_stations):
    dist_km, dist_deg, azimuth = converter.geographic_to_distance(
        center_lat, center_lon, lat, lon
    )

    station_name = f"ARR{i+1:02d}"
    array_info.append({
        'name': station_name,
        'lat': lat,
        'lon': lon,
        'distance': dist_km,
        'azimuth': azimuth
    })

    print(f"{station_name:<8} {lat:<10.3f} {lon:<11.3f} {dist_km:<13.1f} {azimuth:<12.1f}")

In [ ]:
# Analyze array performance for different earthquake scenarios
test_earthquakes = [
    {'lat': 37.5, 'lon': -122.0, 'depth': 5, 'name': 'Central'},
    {'lat': 37.6, 'lon': -121.9, 'depth': 10, 'name': 'Northeast'},
    {'lat': 37.4, 'lon': -122.1, 'depth': 15, 'name': 'Southwest'},
    {'lat': 37.5, 'lon': -121.8, 'depth': 8, 'name': 'East'},
]

print(f"\nArray Performance Analysis:")
print(f"Testing {len(test_earthquakes)} earthquake scenarios...\n")

calc = TravelTimeCalculator('iasp91')
array_performance = {}

for eq in test_earthquakes:
    print(f"Earthquake: {eq['name']} ({eq['lat']:.2f}°N, {abs(eq['lon']):.2f}°W, {eq['depth']} km)")
    print(f"{'Station':<8} {'Distance (km)':<12} {'P-time (s)':<12} {'S-time (s)':<12} {'S-P (s)':<10}")
    print("-" * 60)

    eq_data = []

    for station in array_info:
        # Calculate distance to earthquake
        dist_km, dist_deg, azimuth = converter.geographic_to_distance(
            eq['lat'], eq['lon'], station['lat'], station['lon']
        )

        # Calculate travel times
        arrivals = calc.calculate_travel_times(eq['depth'], dist_deg)

        # Find P and S arrivals
        p_time = None
        s_time = None

        for arrival in arrivals:
            if arrival.name == 'P' and p_time is None:
                p_time = arrival.time
            elif arrival.name == 'S' and s_time is None:
                s_time = arrival.time

        sp_time = s_time - p_time if (s_time and p_time) else None

        eq_data.append({
            'station': station['name'],
            'distance': dist_km,
            'p_time': p_time,
            's_time': s_time,
            'sp_time': sp_time
        })

        print(f"{station['name']:<8} {dist_km:<12.1f} {p_time:<12.2f} {s_time:<12.2f} {sp_time:<10.2f}")

    # Calculate array statistics
    distances = [d['distance'] for d in eq_data]
    p_times = [d['p_time'] for d in eq_data]

    print(f"\nArray Statistics:")
    print(f"  Distance range: {np.min(distances):.1f} - {np.max(distances):.1f} km")
    print(f"  P-time range: {np.min(p_times):.2f} - {np.max(p_times):.2f} s")
    print(f"  P-time spread: {np.max(p_times) - np.min(p_times):.2f} s")
    print(f"  Azimuthal coverage: Good (circular array)\n")

    array_performance[eq['name']] = eq_data

In [ ]:
# Visualize array design and performance
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Array geometry
ax = axes[0, 0]
# Plot array stations
for station in array_info:
    ax.plot(station['lon'], station['lat'], 'g^', markersize=10)
    ax.annotate(station['name'], (station['lon'], station['lat']),
               xytext=(3, 3), textcoords='offset points', fontsize=8)

# Plot test earthquakes
colors_eq = ['red', 'blue', 'green', 'purple']
for eq, color in zip(test_earthquakes, colors_eq):
    ax.plot(eq['lon'], eq['lat'], 'o', color=color, markersize=8, label=eq['name'])

# Plot array circle
array_circle = Circle((center_lon, center_lat), array_radius/111.0,
                     fill=False, color='gray', linestyle='--', alpha=0.7)
ax.add_patch(array_circle)

ax.set_xlabel('Longitude (degrees)')
ax.set_ylabel('Latitude (degrees)')
ax.set_title('Station Array Design')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')

# Travel time analysis for each earthquake
for i, (eq, color) in enumerate(zip(test_earthquakes, colors_eq)):
    if i < 3:  # Plot first 3 earthquakes
        row = (i + 1) // 2
        col = (i + 1) % 2
        ax = axes[row, col]

        eq_data = array_performance[eq['name']]
        stations_names = [d['station'] for d in eq_data]
        p_times = [d['p_time'] for d in eq_data]
        s_times = [d['s_time'] for d in eq_data]

        x_pos = np.arange(len(stations_names))

        ax.bar(x_pos - 0.2, p_times, 0.4, label='P-wave', color='blue', alpha=0.7)
        ax.bar(x_pos + 0.2, s_times, 0.4, label='S-wave', color='red', alpha=0.7)

        ax.set_xlabel('Station')
        ax.set_ylabel('Travel Time (s)')
        ax.set_title(f'Travel Times: {eq["name"]} Earthquake')
        ax.set_xticks(x_pos)
        ax.set_xticklabels(stations_names, rotation=45)
        ax.legend()
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Quality Control and Data Validation

Let's demonstrate how to use seisray for quality control of seismic arrival times.

In [ ]:
# Simulate realistic arrival time data with some outliers
np.random.seed(42)  # For reproducible results

# Reference earthquake
ref_eq_lat = 37.5
ref_eq_lon = -122.0
ref_eq_depth = 12

# Simulate observations from a regional network
observed_data = []
station_coords = [
    {'name': 'STA01', 'lat': 37.8, 'lon': -122.3, 'quality': 'good'},
    {'name': 'STA02', 'lat': 37.2, 'lon': -121.7, 'quality': 'good'},
    {'name': 'STA03', 'lat': 37.6, 'lon': -121.5, 'quality': 'good'},
    {'name': 'STA04', 'lat': 37.9, 'lon': -122.1, 'quality': 'noisy'},  # Noisy station
    {'name': 'STA05', 'lat': 37.1, 'lon': -122.4, 'quality': 'good'},
    {'name': 'STA06', 'lat': 37.7, 'lon': -121.8, 'quality': 'outlier'},  # Bad pick
]

calc = TravelTimeCalculator('iasp91')
converter = CoordinateConverter()

print(f"Quality Control Analysis:")
print(f"Reference earthquake: {ref_eq_lat:.2f}°N, {abs(ref_eq_lon):.2f}°W, {ref_eq_depth} km")
print(f"\n{'Station':<8} {'Dist(km)':<8} {'Predicted':<10} {'Observed':<10} {'Residual':<10} {'Quality':<8}")
print("-" * 70)

for station in station_coords:
    # Calculate true distance and predicted travel time
    dist_km, dist_deg, azimuth = converter.geographic_to_distance(
        ref_eq_lat, ref_eq_lon, station['lat'], station['lon']
    )

    arrivals = calc.calculate_travel_times(ref_eq_depth, dist_deg)
    predicted_p = None
    for arrival in arrivals:
        if arrival.name == 'P':
            predicted_p = arrival.time
            break

    # Simulate observed time with appropriate noise/errors
    if station['quality'] == 'good':
        noise = np.random.normal(0, 0.1)  # ±0.1 s standard error
    elif station['quality'] == 'noisy':
        noise = np.random.normal(0, 0.3)  # ±0.3 s standard error
    elif station['quality'] == 'outlier':
        noise = np.random.normal(2.0, 0.2)  # Systematic +2s error (bad pick)

    observed_p = predicted_p + noise
    residual = observed_p - predicted_p

    observed_data.append({
        'station': station['name'],
        'distance': dist_km,
        'predicted': predicted_p,
        'observed': observed_p,
        'residual': residual,
        'quality': station['quality']
    })

    print(f"{station['name']:<8} {dist_km:<8.1f} {predicted_p:<10.2f} {observed_p:<10.2f} {residual:<10.2f} {station['quality']:<8}")

In [ ]:
# Quality control analysis
residuals = np.array([d['residual'] for d in observed_data])
distances = np.array([d['distance'] for d in observed_data])
stations = [d['station'] for d in observed_data]

# Statistical analysis
mean_residual = np.mean(residuals)
std_residual = np.std(residuals)
rms_residual = np.sqrt(np.mean(residuals**2))

print(f"\nQuality Control Statistics:")
print(f"  Mean residual: {mean_residual:.3f} s")
print(f"  Standard deviation: {std_residual:.3f} s")
print(f"  RMS residual: {rms_residual:.3f} s")

# Outlier detection (3-sigma rule)
outlier_threshold = 3 * std_residual
outliers = np.abs(residuals) > outlier_threshold

print(f"\nOutlier Detection (3σ rule):")
print(f"  Threshold: ±{outlier_threshold:.3f} s")
print(f"  Outliers detected: {np.sum(outliers)} stations")

if np.any(outliers):
    print(f"  Outlier stations:")
    for i, is_outlier in enumerate(outliers):
        if is_outlier:
            print(f"    {stations[i]}: {residuals[i]:.3f} s")

# Alternative robust statistics (median-based)
median_residual = np.median(residuals)
mad_residual = np.median(np.abs(residuals - median_residual))  # Median Absolute Deviation
robust_outliers = np.abs(residuals - median_residual) > 3 * mad_residual

print(f"\nRobust Outlier Detection (Median + MAD):")
print(f"  Median residual: {median_residual:.3f} s")
print(f"  MAD: {mad_residual:.3f} s")
print(f"  Robust outliers: {np.sum(robust_outliers)} stations")

In [ ]:
# Visualize quality control results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Residuals vs distance
ax = axes[0, 0]
colors = ['green' if q == 'good' else 'orange' if q == 'noisy' else 'red'
          for q in [d['quality'] for d in observed_data]]
scatter = ax.scatter(distances, residuals, c=colors, s=80, alpha=0.7)
ax.axhline(0, color='black', linestyle='-', alpha=0.5)
ax.axhline(outlier_threshold, color='red', linestyle='--', alpha=0.7, label=f'3σ = ±{outlier_threshold:.2f}s')
ax.axhline(-outlier_threshold, color='red', linestyle='--', alpha=0.7)
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Travel Time Residual (s)')
ax.set_title('Residuals vs Distance')
ax.legend()
ax.grid(True, alpha=0.3)

# Residual histogram
ax = axes[0, 1]
ax.hist(residuals, bins=10, alpha=0.7, color='skyblue', edgecolor='black')
ax.axvline(mean_residual, color='red', linestyle='-', linewidth=2, label=f'Mean = {mean_residual:.3f}s')
ax.axvline(median_residual, color='green', linestyle='-', linewidth=2, label=f'Median = {median_residual:.3f}s')
ax.axvline(outlier_threshold, color='red', linestyle='--', alpha=0.7)
ax.axvline(-outlier_threshold, color='red', linestyle='--', alpha=0.7)
ax.set_xlabel('Travel Time Residual (s)')
ax.set_ylabel('Frequency')
ax.set_title('Residual Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Travel time curves
ax = axes[1, 0]
predicted_times = [d['predicted'] for d in observed_data]
observed_times = [d['observed'] for d in observed_data]

ax.scatter(distances, predicted_times, color='blue', s=60, label='Predicted', alpha=0.7)
ax.scatter(distances, observed_times, c=colors, s=60, label='Observed', alpha=0.7)

# Connect predicted and observed for each station
for i in range(len(distances)):
    ax.plot([distances[i], distances[i]], [predicted_times[i], observed_times[i]],
           'k-', alpha=0.3, linewidth=1)

ax.set_xlabel('Distance (km)')
ax.set_ylabel('Travel Time (s)')
ax.set_title('Predicted vs Observed Travel Times')
ax.legend()
ax.grid(True, alpha=0.3)

# Station quality summary
ax = axes[1, 1]
station_names = [d['station'] for d in observed_data]
station_residuals = [d['residual'] for d in observed_data]
station_colors = ['green' if abs(r) <= outlier_threshold else 'red' for r in station_residuals]

bars = ax.bar(range(len(station_names)), station_residuals, color=station_colors, alpha=0.7)
ax.axhline(0, color='black', linestyle='-', alpha=0.5)
ax.axhline(outlier_threshold, color='red', linestyle='--', alpha=0.7)
ax.axhline(-outlier_threshold, color='red', linestyle='--', alpha=0.7)
ax.set_xlabel('Station')
ax.set_ylabel('Travel Time Residual (s)')
ax.set_title('Station Quality Assessment')
ax.set_xticks(range(len(station_names)))
ax.set_xticklabels(station_names, rotation=45)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Quality summary
good_stations = [d['station'] for d in observed_data if abs(d['residual']) <= outlier_threshold]
bad_stations = [d['station'] for d in observed_data if abs(d['residual']) > outlier_threshold]

print(f"\nFinal Quality Assessment:")
print(f"  Good stations ({len(good_stations)}): {', '.join(good_stations)}")
print(f"  Problematic stations ({len(bad_stations)}): {', '.join(bad_stations)}")
print(f"  Overall data quality: {len(good_stations)}/{len(observed_data)} stations passed QC")

## Summary

In this notebook, we demonstrated practical applications of the `seisray` package:

1. **Earthquake Location**: 
   - Simulated realistic source-receiver geometries
   - Generated synthetic arrival times with noise
   - Analyzed station network coverage

2. **Regional Velocity Structure Studies**:
   - Compared different Earth models for regional distances
   - Quantified model differences and uncertainties
   - Assessed model performance for specific regions

3. **Station Array Design**:
   - Designed optimal circular station arrays
   - Analyzed array performance for different earthquake scenarios
   - Evaluated azimuthal coverage and timing resolution

4. **Quality Control and Data Validation**:
   - Implemented statistical outlier detection methods
   - Compared predicted vs observed travel times
   - Assessed data quality and station performance

### Key Practical Insights:

**Earthquake Location:**
- Good azimuthal coverage is essential for accurate locations
- S-P times provide additional constraints for depth estimation
- Regional networks typically achieve ~1-5 km location accuracy

**Model Selection:**
- Model differences are typically <1 second for regional distances
- IASP91 remains adequate for most regional studies
- Local velocity models may be needed for high-precision work

**Array Design:**
- Circular arrays provide optimal azimuthal coverage
- Array aperture should match expected source distances
- 6-12 stations often provide good balance of cost vs. performance

**Quality Control:**
- Outlier detection is crucial for reliable analysis
- 3σ rule catches most obvious errors
- Robust statistics help with non-Gaussian error distributions
- Systematic station biases may indicate calibration issues

### Applications in Research:
- **Seismic monitoring networks**: Design and optimization
- **Regional tomography**: Data quality assessment
- **Earthquake early warning**: Network performance analysis
- **Induced seismicity studies**: Local network design
- **Educational purposes**: Understanding seismic wave propagation

The `seisray` package provides essential tools for practical seismological applications, from research planning to data analysis and quality control.